In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def create_schedule_matrix():
    """Crea una matriz de horario vacía (5 días x 9 bloques)."""
    return [[False for _ in range(9)] for _ in range(5)]

def get_day_index(day):
    """Convierte el nombre del día a índice."""
    days = {'Lunes': 0, 'Martes': 1, 'Miercoles': 2, 'Jueves': 3, 'Viernes': 4}
    return days.get(day, -1)

def find_windows_in_day(day_schedule):
    """
    Encuentra ventanas (huecos) en un día específico.
    Retorna la cantidad de ventanas y su duración total.
    """
    windows = 0
    total_duration = 0
    
    # Encontrar el primer y último bloque ocupado
    first_occupied = -1
    last_occupied = -1
    for i, block in enumerate(day_schedule):
        if block:
            if first_occupied == -1:
                first_occupied = i
            last_occupied = i
    
    if first_occupied == -1:  # No hay bloques ocupados
        return 0, 0
    
    # Contar ventanas entre el primer y último bloque ocupado
    in_window = False
    current_window_length = 0
    
    for i in range(first_occupied, last_occupied + 1):
        if not day_schedule[i]:  # Bloque libre
            current_window_length += 1
            in_window = True
        else:  # Bloque ocupado
            if in_window:
                windows += 1
                total_duration += current_window_length
                current_window_length = 0
                in_window = False
    
    return windows, total_duration

def analyze_room_compaction(room_data):
    """
    Analiza la compactación del horario de una sala específica.
    Retorna la cantidad total de ventanas y su duración total.
    """
    schedule = create_schedule_matrix()
    
    # Marcar bloques ocupados
    for class_data in room_data['Asignaturas']:
        day_idx = get_day_index(class_data['Dia'])
        if day_idx != -1:
            block_idx = class_data['Bloque'] - 1
            if 0 <= block_idx < 9:
                schedule[day_idx][block_idx] = True
    
    # Analizar ventanas por día
    total_windows = 0
    total_duration = 0
    for day_schedule in schedule:
        windows, duration = find_windows_in_day(day_schedule)
        total_windows += windows
        total_duration += duration
    
    return {
        'total_windows': total_windows,
        'total_duration': total_duration,
        'schedule': schedule
    }

def create_summary_table(horarios_salas):
    """Crea una tabla resumen de compactación para todas las salas."""
    summary_data = []
    
    for sala in horarios_salas:
        stats = analyze_room_compaction(sala)
        summary_data.append({
            'Codigo': sala['Codigo'],
            'Cantidad de Ventanas': stats['total_windows'],
            'Duracion de Ventanas': stats['total_duration']
        })
    df_summary = pd.DataFrame(summary_data)

    # Añadir fila de totales
    total_row = pd.DataFrame([{
        'Codigo': 'Total',
        'Cantidad de Ventanas': df_summary['Cantidad de Ventanas'].sum(),
        'Duracion de Ventanas': df_summary['Duracion de Ventanas'].sum()
    }])
    df_summary = pd.concat([df_summary, total_row], ignore_index=True)
    
    
    # Guardar tabla en CSV
    df_summary.to_csv('resumen_compactacion.csv', index=False)
    
    # Mostrar tabla formateada
    print("\nResumen de Compactación por Sala:")
    print(df_summary.to_string(index=False))
    
    return df_summary

def create_scatter_plot(df_summary):
    """Crea un gráfico de dispersión de ventanas vs duración."""
    plt.figure(figsize=(12, 8))
    
    plt.scatter(df_summary['Cantidad de Ventanas'], 
               df_summary['Duracion de Ventanas'],
               alpha=0.6)
    
    # Añadir etiquetas para cada punto
    for idx, row in df_summary.iterrows():
        plt.annotate(row['Codigo'], 
                    (row['Cantidad de Ventanas'], row['Duracion de Ventanas']),
                    xytext=(5, 5), textcoords='offset points')
    
    plt.title('Relación entre Cantidad y Duración de Ventanas por Sala')
    plt.xlabel('Cantidad de Ventanas')
    plt.ylabel('Duración Total de Ventanas (bloques)')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Añadir línea de tendencia
    z = np.polyfit(df_summary['Cantidad de Ventanas'], 
                  df_summary['Duracion de Ventanas'], 1)
    p = np.poly1d(z)
    plt.plot(df_summary['Cantidad de Ventanas'], 
             p(df_summary['Cantidad de Ventanas']), 
             "r--", alpha=0.8)
    
    plt.tight_layout()
    plt.savefig('compactacion_scatter.png')
    plt.close()

def save_results(df_summary):
    """Guarda los resultados en un archivo JSON."""
    try:
        results = {
            'room_stats': df_summary.to_dict(orient='records'),
            'global_stats': {
                'promedio_ventanas': float(df_summary['Cantidad de Ventanas'].mean()),
                'promedio_duracion': float(df_summary['Duracion de Ventanas'].mean()),
                'max_ventanas': int(df_summary['Cantidad de Ventanas'].max()),
                'max_duracion': int(df_summary['Duracion de Ventanas'].max())
            }
        }
        
        with open('metricas_compactacion.json', 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        return True
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    try:
        with open('../../agent_output/Horarios_salas.json', 'r', encoding='utf-8') as file:
            horarios_salas = json.load(file)
    except Exception as e:
        print(f"Error al cargar el archivo: {str(e)}")
        return
    
    # 2. Crear tabla resumen
    df_summary = create_summary_table(horarios_salas)
    if df_summary is None:
        return
    
    # 3. Crear gráfico de dispersión
    create_scatter_plot(df_summary)
    
    # 4. Guardar resultados
    if save_results(df_summary):
        print("\nAnálisis de compactación completado exitosamente")
    else:
        print("Error al completar el análisis de compactación")

if __name__ == "__main__":
    main()


Resumen de Compactación por Sala:
Codigo  Cantidad de Ventanas  Duracion de Ventanas
 KAUS3                     7                     7
 KAUS2                    15                    15
 KAUS1                    10                    10
    K1                    11                    13
    K2                    15                    15
  A106                     0                     0
   CM3                    13                    13
    K3                    11                    11
   CM5                    10                    10
   CM4                    10                    10
    E1                     8                    12
   CM6                     8                    14
   IM3                     8                    12
   IM2                     7                    11
   IM4                    10                    12
    E6                     7                    11
   IC1                    11                    11
 CRP43                    10                   